### Consulta 1
def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.


In [1]:
import pandas as pd

In [2]:
columns = ['id', 'price']
games_price = pd.read_csv(r'./datasets/steam_games.csv', usecols=columns)
games_price.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [3]:
games_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [4]:
unique_list = []
for i in games_price['price']:
    try:
        float(i)
    except:
        unique_list.append(i)

uniques = set(unique_list)
uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']

In [5]:
print(uniques)

{'Play for Free!', 'Free To Play', 'Play Now', 'Free Movie', 'Free Demo', 'Play the Demo', 'Starting at $449.00', 'Free to Use', 'Free to Play', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Third-party', 'Free to Try', 'Free HITMAN™ Holiday Pack', 'Starting at $499.00', 'Free Mod', 'Install Theme', 'Free'}


In [6]:
games_price['price_fixed'] = games_price['price'].apply(lambda x: 0 if x in uniques 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
games_price.head()

,price,id,price_fixed
0,4.99,761140.0,4.99
1,Free To Play,643980.0,0
2,Free to Play,670290.0,0
3,0.99,767400.0,0.99
4,2.99,773570.0,2.99


In [7]:
games_price['price_fixed'] = games_price['price_fixed'].astype(float)

In [8]:
games_price.rename(columns={'id':'item_id'},inplace=True)
games_price.drop(columns='price', inplace=True)
games_price.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [9]:
games_price.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [10]:
games_price.fillna(0, inplace=True)

In [11]:
games_price.isna().sum()

item_id        0
price_fixed    0
dtype: int64

In [12]:
columns = ['user_id', 'item_id', 'items_count']
user_items = pd.read_csv('datasets/aus_users_items.csv', usecols=columns)
user_items.head()

,user_id,items_count,item_id
0,76561197970982479,277,10
1,76561197970982479,277,20
2,76561197970982479,277,30
3,76561197970982479,277,40
4,76561197970982479,277,50


In [13]:
df_merged = user_items.merge(games_price, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
4131569,jaeger18,39,326460,6.99
594169,piepai,6132,392060,NaN
1816251,aaaarrrghhhh,60,94200,9.99
4978693,76561198086761037,9,109600,0.00
1610081,76561198066562072,160,35420,0.00
1856133,B3nd0rz,443,40800,14.99
853411,FDAOP,468,228200,NaN
786555,jaydosM8,73,409720,19.99
2430288,J4K3_PL4Y5,44,269390,NaN
4168079,76561198032003610,1075,201420,14.99


In [14]:
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
user_spent = pd.DataFrame(user_spent)
user_spent.reset_index()

,user_id,price_fixed
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


In [15]:
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('./datasets/aus_user_revs_clean.csv', usecols=columns)



In [16]:
user_revs.sample(5)

,user_id,recommend
22176,76561198067662523,True
30538,decnics,True
35628,imdagiver,True
29556,195862,True
226,76561198073784601,False


In [17]:
user_gb_count_revs = pd.DataFrame()

user_gb_count_revs['count_total'] = user_revs.groupby('user_id')['recommend'].agg('count')
user_gb_count_revs['count_true'] = user_revs[user_revs['recommend'] == True].groupby('user_id')['recommend'].agg(count_revs='count')


user_gb_count_revs.reset_index(inplace=True)
user_gb_count_revs

# proporcion
user_gb_count_revs['reco_porcentaje'] = user_gb_count_revs['count_true'] / user_gb_count_revs['count_total']
user_gb_count_revs.fillna(0,inplace=True)

In [18]:
user_gb_count_revs.sample(5)

,user_id,count_total,count_true,reco_porcentaje
17255,That_Chap,2,1.0,0.5
13867,GGargula,2,1.0,0.5
18526,athena8987,2,2.0,1.0
19950,eledga,3,3.0,1.0
2700,76561198042503321,1,1.0,1.0


In [19]:
user_revs[user_revs['user_id']=='iliakitmail']

,user_id,recommend
27618,iliakitmail,False


In [151]:
user_items.

,user_id,items_count,item_id
0,76561197970982479,277,10
1,76561197970982479,277,20
2,76561197970982479,277,30
3,76561197970982479,277,40
4,76561197970982479,277,50


In [ ]:
df_merge2 = user_gb_count_revs.merge(user_spent, on='user_id',how='left' )
df_merge2.rename(columns={'price_fixed':'total_spent'},inplace=True)


In [37]:
df_merge3 = df_merge2.merge(user_items[['user_id','items_count']].drop_duplicates(),on='user_id',how='left')
df_merge3 = df_merge3.drop(['count_total','count_true'],axis=1)
df_merge3['reco_porcentaje'] = df_merge3['reco_porcentaje'].apply(lambda x : int(x *100))
df_merge3

,user_id,reco_porcentaje,total_spent,items_count
0,--000--,100,402.77,58.0
1,--ace--,100,166.82,44.0
2,--ionex--,100,109.92,23.0
3,-2SV-vuLB-Kg,100,437.49,68.0
4,-Azsael-,100,2461.68,167.0
...,...,...,...,...
25453,zwanzigdrei,100,913.44,93.0
25454,zy0705,100,0.00,6.0
25455,zynxgameth,100,315.09,50.0
25456,zyr0n1c,100,667.56,73.0


In [39]:
df_merge3.to_csv(r'dataquery/user_data.csv',index=False)